# II Stratified Sampling
In this notebook, we will attempt to price options on an asset S governed by geometric brownian motion $dS(t) = S(t)dt + \sigma S(t) dW(t)$, where r and $\sigma$ are constant. We will focus on developing a method for stratified sampling for variance reduction, and comparing the performance of this method with standard Monte Carlo and other variance reduction techniques.

## Introduction to methodology
We can easily create stratified paths of Brownian motion using the method outlined in the notes. To convert these to paths of geometric Brownian motion, we can use the equation
$$S_t = S_0 \exp((r - \frac{1}{2} \sigma^2) t + \sigma W_t)$$

***
### Import libraries and set up parameters

In [1]:
# Import libraries
import numpy as np
from BlackScholes import BS_call
from scipy.stats import norm

In [2]:
# Set parameters
Npaths = 10 ** 4
S0 = 100
K=100
T=2
r=0.05
sigma=0.2
strata = 40

In [3]:
# Construct a random generator 
rng = np.random.default_rng(12345)

***
## Geometric Brownian motion with stratified sampling
We begin by constructing a function that creates a Brownian bridge, to use in generating stratified paths of Brownian motion. After this, we create a function which outputs the stratified paths of Brownian motion, before finally turning these into geometric Brownian motion.

In [4]:
def Brownbridge(B0, BT, T, M, Npaths):
    
    # Generates Npaths Brownian bridges from B0 to BT using M levels
    # return a time grid t and the Brownian bridges B
    
    # Set N, the time grid, and the array for B 
    N = 2**M
    t = np.linspace(0, T, N+1)
    B = np.zeros((N+1, Npaths))
    
    # set initial and final values of B
    B[0, :] = B0
    B[N, :] = BT

    # set initial time spacing and grid spacing
    delta_T = T
    delta_n = N

    # loop over levels
    for m in range(M):

        # loop over the 2**m grid points in level m
        for k in range(2**m):
            n = k * delta_n
            B[n + delta_n//2,:] = 0.5*(B[n,:]+B[n+delta_n,:]) + np.sqrt(delta_T/4) * rng.normal(size = Npaths)

        # halve delta_T and delta_n before going to next level
        delta_T = delta_T/2
        delta_n = delta_n//2     
    
    return B

def stratified_GBM(S0, T, r, sigma, Npaths, strata, log2_timesteps):
    Nsteps = 2 ** log2_timesteps
    Nstrat = int(Npaths / strata)
    trueNpaths = Nstrat * strata # By doing this we should be more robust to bad inputs
    
    # We use a 3D array W_ijk where i is the relevant strata, j is time and k is the path
    # After this is simple Euler time stepping
    W = np.zeros((strata, Nsteps + 1, Nstrat))
    t, dt = np.linspace(0, T, Nsteps + 1, retstep=True)
    
    for m in range(strata):
        Y = rng.uniform(m / strata, (m + 1) / strata, Nstrat)
        WT = np.sqrt(T) * norm.ppf(Y)
        W[m, :, :] = Brownbridge(0, WT, T, log2_timesteps, Nstrat)
    
    # Now work out stratified GBM using the stratified BM above
    S = 0 * W
    for n in range(Nsteps + 1):
        S[:, n, :] = S0 * np.exp((r - 0.5 * sigma ** 2) * t[n] + sigma * W[:, n, :])
    
    return t, S

***
## Option pricing using stratified sampling
We will now price European, Asian, and Lookback call options using stratified sampling, as well as comparing it to other approaches, such as naive Monte Carlo. We use path dependent calculations for all comparisons (including European options) to maintain a fair comparison. We have hidden the code cells for naive and antithetic Monte Carlo as they are not relevant to the project.

***
### European call options
We begin by considering European call options, with payoff given by $\max(S(T) - K, 0)$:

In [5]:
def MC_strat_euro_call(S0, K, T, r, sigma, Npaths, strata):
    Nsteps_approx = T * 260
    log2_timesteps = int(np.log(Nsteps_approx) / np.log(2)) # approximately 260 trading days / year
    
    t, S = stratified_GBM(S0, T, r, sigma, Npaths, strata, log2_timesteps)
    
    # standard European payoff calculation
    ST = S[:, -1, :]
    fST = np.exp(-r * T) * np.maximum(ST - K, 0)
    
    return np.mean(np.mean(fST, axis = 1)), np.mean(np.var(fST, axis = 1))

In [6]:
MC_strat_euro = MC_strat_euro_call(S0, K, T, r, sigma, Npaths, strata)
SEM = np.sqrt(MC_strat_euro[1]/Npaths) # Use standard error of mean to approximate 95% interval

BS_euro = BS_call(S0, K, T, r, sigma)

print("Stratified European call price =", round(MC_strat_euro[0],4), "+/-", format(1.96*SEM,'.2g'))
print("BS European call price =", round(BS_euro,4))

Stratified European call price = 16.1453 +/- 0.07
BS European call price = 16.1268


We were able to compare the output to the true value of European call options as the European options have an explicit solution. This will not be true for the other two options.

We will now compare the above to the results of naive Monte Carlo and antithetic variance reduction, as well as the required accuracy required to reach an absolute error below 0.05 to quantify performance.

In [7]:
# Functions for naive monte carlo and antithetic variates
def SDE_euro_call(S0, K, T, r, sigma, Npaths):
    
    # price European call option by simulating GBM paths
    # returns the price and variance
    
    # Use a time step of 1 day assuming 260 days/year
    Nsteps = int(260 * T)
    
    t, dt = np.linspace(0, T, Nsteps + 1, retstep=True)
    S = np.zeros((Nsteps+1, Npaths))
    
    # Time step starting from initial condition
    S[0,:] = S0  
    for n in range(Nsteps):
        dW = np.sqrt(dt) * rng.normal(0,1,Npaths)
        S[n+1,:] = S[n,:] * (1. + r*dt + sigma * dW[:])
    
    # discounted payoff based on S at final time
    fST = np.exp(-r*T)*np.maximum(S[-1,:]-K,0)

    # compute and return price and variance
    price = np.mean(fST)
    variance = np.var(fST)
    return price, variance

def SDE_euro_call_ant(S0, K, T, r, sigma, Npaths):
    
    # price European call option by simulating GBM paths
    # with antithetic variance reduction
    # returns the price and variance
    
    # Use a time step of 1 day assuming 260 days/year
    Nsteps = int(260 * T)
    
    t, dt = np.linspace(0, T, Nsteps + 1, retstep=True)
    Sp = np.zeros((Nsteps+1, Npaths))
    Sm = np.zeros((Nsteps+1, Npaths))
    
    # Time step starting from initial condition
    Sp[0,:] = S0 
    Sm[0,:] = S0 
    for n in range(Nsteps):
        dW = np.sqrt(dt) * rng.normal(0,1,Npaths)
        Sp[n+1,:] = Sp[n,:] * (1. + r*dt + sigma * dW[:])
        Sm[n+1,:] = Sm[n,:] * (1. + r*dt - sigma * dW[:])

    # discounted payoffs based on Sp, Sm at final time
    fSTp = np.exp(-r*T)*np.maximum(Sp[-1,:]-K,0)
    fSTm = np.exp(-r*T)*np.maximum(Sm[-1,:]-K,0)

    # compute and return price and variance
    Z = (fSTp + fSTm)/2
    price = np.mean(Z)
    variance = np.var(Z)
    return price, variance

In [8]:
MC_euro = SDE_euro_call(S0, K, T, r, sigma, Npaths)
SEM = np.sqrt(MC_euro[1]/Npaths) # Use standard error of mean to approximate 95% interval

print("Naive European call price =", round(MC_euro[0],4), "+/-", format(1.96*SEM,'.2g'))

MC_euro_ant = SDE_euro_call_ant(S0, K, T, r, sigma, Npaths)
SEM = np.sqrt(MC_euro_ant[1]/Npaths) # Use standard error of mean to approximate 95% interval

print("Antithetic European call price =", round(MC_euro_ant[0],4), "+/-", format(1.96*SEM,'.2g'))

Naive European call price = 15.8797 +/- 0.44
Antithetic European call price = 16.178 +/- 0.22


In [9]:
error = 0.05

N_naive = int((1.96/error)**2 * MC_euro[1])
N_ant = int((1.96/error)**2 * MC_euro_ant[1])
N_strat = int((1.96/error)**2 * MC_strat_euro[1])

print("The required number of samples for an absolute error below 0.05 for European call options is:\n",
      f"{N_naive} for naive Monte Carlo \n",
      f"{N_ant} with antithetic variable variance reduction \n",
      f"{N_strat} with stratified GBM paths")

The required number of samples for an absolute error below 0.05 for European call options is:
 767509 for naive Monte Carlo 
 189697 with antithetic variable variance reduction 
 19737 with stratified GBM paths


We see that there is a dramatic increase in performance for stratified sampling, compared to naive Monte Carlo and antithetic variates.

***
### Asian call options
We now consider Asian call options, with payoff given by $\max(\bar{S} - K, 0)$, where $\bar{S}$ is the average value of the asset $S(t)$ over the life of the option $0 \leq t \leq T$:

In [10]:
def MC_strat_asian_call(S0, K, T, r, sigma, Npaths, strata):
    Nsteps_approx = T * 260
    log2_timesteps = int(np.log(Nsteps_approx) / np.log(2)) # approximately 260 trading days / year
    
    t, S = stratified_GBM(S0, T, r, sigma, Npaths, strata, log2_timesteps)
    
    average_values = np.mean(S, axis = 1)
    fST = np.exp(-r * T) * np.maximum(average_values - K, 0)
    
    return np.mean(np.mean(fST, axis = 1)), np.mean(np.var(fST, axis = 1))

In [11]:
MC_strat_asian = MC_strat_asian_call(S0, K, T, r, sigma, Npaths, strata)
SEM = np.sqrt(MC_strat_asian[1]/Npaths) # Use standard error of mean to approximate 95% interval

print("Stratified Asian call price =", round(MC_strat_asian[0],4), "+/-", format(1.96*SEM,'.2g'))

Stratified Asian call price = 8.7331 +/- 0.12


As before, we now compare the above to the results of naive Monte Carlo and antithetic variance reduction, as well as the required accuracy required to reach an absolute error below 0.05 to quantify performance.

In [12]:
def SDE_asian_call(S0, K, T, r, sigma, Npaths):
    
    # price Asian call option by simulating GBM paths
    # returns the price and variance
    
    # Use a time step of 1 day assuming 260 days/year
    Nsteps = int(260 * T)
    
    t, dt = np.linspace(0, T, Nsteps + 1, retstep=True)
    S = np.zeros((Nsteps+1, Npaths))
    
    # Time step starting from initial condition
    S[0,:] = S0  
    for n in range(Nsteps):
        dW = np.sqrt(dt) * rng.normal(0,1,Npaths)
        S[n+1,:] = S[n,:] * (1. + r*dt + sigma * dW[:])
        
    # for Asian call
    fS = np.exp(-r*T)*np.maximum(np.mean(S,axis=0) - K, 0)

    price = np.mean(fS)
    variance = np.var(fS)
    return price, variance

def SDE_asian_call_ant(S0, K, T, r, sigma, Npaths):
    
    # price Asian call option by simulating GBM paths
    # returns the price and variance
    
    # Use a time step of 1 day assuming 260 days/year
    Nsteps = int(260 * T)
    
    t, dt = np.linspace(0, T, Nsteps + 1, retstep=True)
    Sp = np.zeros((Nsteps+1, Npaths))
    Sm = np.zeros((Nsteps+1, Npaths))
    
    # Time step starting from initial condition
    Sp[0,:] = S0
    Sm[0,:] = S0
    for n in range(Nsteps):
        dW = np.sqrt(dt) * rng.normal(0,1,Npaths)
        Sp[n+1,:] = Sp[n,:] * (1. + r*dt + sigma * dW[:])
        Sm[n+1,:] = Sm[n,:] * (1. + r*dt - sigma * dW[:])
        
    # for Asian call
    fSp = np.exp(-r*T)*np.maximum(np.mean(Sp,axis=0) - K, 0)
    fSm = np.exp(-r*T)*np.maximum(np.mean(Sm,axis=0) - K, 0)
    
    Z = (fSp + fSm)/2
    
    price = np.mean(Z)
    variance = np.var(Z)
    return price, variance

In [13]:
MC_asian = SDE_asian_call(S0, K, T, r, sigma, Npaths)
SEM = np.sqrt(MC_asian[1]/Npaths) # Use standard error of mean to approximate 95% interval

print("Naive Asian call price =", round(MC_asian[0],4), "+/-", format(1.96*SEM,'.2g'))

MC_asian_ant = SDE_asian_call_ant(S0, K, T, r, sigma, Npaths)
SEM = np.sqrt(MC_asian_ant[1]/Npaths) # Use standard error of mean to approximate 95% interval

print("Antithetic Asian call price =", round(MC_asian_ant[0],4), "+/-", format(1.96*SEM,'.2g'))

Naive Asian call price = 8.9344 +/- 0.23
Antithetic Asian call price = 8.6999 +/- 0.11


In [14]:
error = 0.05

N_naive = int((1.96/error)**2 * MC_asian[1])
N_ant = int((1.96/error)**2 * MC_asian_ant[1])
N_strat = int((1.96/error)**2 * MC_strat_asian[1])

print("The required number of samples for an absolute error below 0.05 for Asian call options is:\n",
      f"{N_naive} for naive Monte Carlo \n",
      f"{N_ant} with antithetic variable variance reduction \n",
      f"{N_strat} with stratified GBM paths")

The required number of samples for an absolute error below 0.05 for Asian call options is:
 216254 for naive Monte Carlo 
 49622 with antithetic variable variance reduction 
 62215 with stratified GBM paths


Here we observe a less dramatic improvement in performance. In fact, our stratified paths are outperformed by antithetic variance reduction.

***
### Lookback call option
Finally, we consider floating-strike lookback call options, with payoff given by $\max(S(T) - D, 0)$ where D is the minimum value of $S(t)$ over the life of the option $0 \leq t \leq T$:

In [15]:
def MC_strat_lookback_call(S0, T, r, sigma, Npaths, strata):
    Nsteps_approx = T * 260
    log2_timesteps = int(np.log(Nsteps_approx) / np.log(2)) # approximately 260 trading days / year
    
    t, S = stratified_GBM(S0, T, r, sigma, Npaths, strata, log2_timesteps)
    
    min_values = np.min(S, axis = 1)
    fST = np.exp(-r * T) * np.maximum(S[:, -1, :] - min_values, 0)
    
    return np.mean(np.mean(fST, axis = 1)), np.mean(np.var(fST, axis = 1))

In [16]:
MC_strat_lookback = MC_strat_lookback_call(S0, T, r, sigma, Npaths, strata)
SEM = np.sqrt(MC_strat_lookback[1]/Npaths) # Use standard error of mean to approximate 95% interval

print("Stratified Lookback call price =", round(MC_strat_lookback[0],4), "+/-", format(1.96*SEM,'.2g'))

Stratified Lookback call price = 24.2932 +/- 0.13


Again, we compare to naive Monte Carlo and Monte Carlo with antithetic variance reduction.

In [17]:
def SDE_lookback_call(S0, T, r, sigma, Npaths):
    
    # price Lookback call option by simulating GBM paths
    # returns the price and variance
    
    # Use a time step of 1 day assuming 260 days/year
    Nsteps = int(260 * T)
    
    t, dt = np.linspace(0, T, Nsteps + 1, retstep=True)
    S = np.zeros((Nsteps+1, Npaths))
    
    # Time step starting from initial condition
    S[0,:] = S0  
    for n in range(Nsteps):
        dW = np.sqrt(dt) * rng.normal(0,1,Npaths)
        S[n+1,:] = S[n,:] * (1. + r*dt + sigma * dW[:])
        
    # for Lookback call
    fS = np.exp(-r*T)*np.maximum(S[-1, :] - np.min(S,axis=0), 0)

    price = np.mean(fS)
    variance = np.var(fS)
    return price, variance

def SDE_lookback_call_ant(S0, T, r, sigma, Npaths):
    
    # price Lookback call option by simulating GBM paths
    # returns the price and variance
    
    # Use a time step of 1 day assuming 260 days/year
    Nsteps = int(260 * T)
    
    t, dt = np.linspace(0, T, Nsteps + 1, retstep=True)
    Sp = np.zeros((Nsteps+1, Npaths))
    Sm = np.zeros((Nsteps+1, Npaths))
    
    # Time step starting from initial condition
    Sp[0,:] = S0
    Sm[0,:] = S0
    for n in range(Nsteps):
        dW = np.sqrt(dt) * rng.normal(0,1,Npaths)
        Sp[n+1,:] = Sp[n,:] * (1. + r*dt + sigma * dW[:])
        Sm[n+1,:] = Sm[n,:] * (1. + r*dt - sigma * dW[:])
        
    # for Lookback call
    fSp = np.exp(-r*T)*np.maximum(Sp[-1, :] - np.min(Sp,axis=0), 0)
    fSm = np.exp(-r*T)*np.maximum(Sm[-1, :] - np.min(Sm,axis=0), 0)
    
    Z = (fSp + fSm)/2
    
    price = np.mean(Z)
    variance = np.var(Z)
    return price, variance

In [18]:
MC_lookback = SDE_lookback_call(S0, T, r, sigma, Npaths)
SEM = np.sqrt(MC_lookback[1]/Npaths) # Use standard error of mean to approximate 95% interval

print("Naive Lookback call price =", round(MC_lookback[0],4), "+/-", format(1.96*SEM,'.2g'))

MC_lookback_ant = SDE_lookback_call_ant(S0, T, r, sigma, Npaths)
SEM = np.sqrt(MC_lookback_ant[1]/Npaths) # Use standard error of mean to approximate 95% interval

print("Antithetic Lookback call price =", round(MC_lookback_ant[0],4), "+/-", format(1.96*SEM,'.2g'))

Naive Lookback call price = 24.1145 +/- 0.44
Antithetic Lookback call price = 24.4356 +/- 0.18


In [19]:
error = 0.05

N_naive = int((1.96/error)**2 * MC_lookback[1])
N_ant = int((1.96/error)**2 * MC_lookback_ant[1])
N_strat = int((1.96/error)**2 * MC_strat_lookback[1])

print("The required number of samples for an absolute error below 0.05 for Lookback call options is:\n",
      f"{N_naive} for naive Monte Carlo \n",
      f"{N_ant} with antithetic variable variance reduction \n",
      f"{N_strat} with stratified GBM paths")

The required number of samples for an absolute error below 0.05 for Lookback call options is:
 765028 for naive Monte Carlo 
 133736 with antithetic variable variance reduction 
 63087 with stratified GBM paths


Here, we observe a large improvement from using stratified sampling, both against naive Monte Carlo, as well as against Monte Carlo with antithetic variates.

***
## Conclusion
We have seen above that using stratified sampling is a powerful way to reduce variance, even for path dependent options. In the case of European and Lookback options, stratified sampling we able to outperform even antithetic variates, but this was not the case for Asian options. When increasing the number of strata (even up to 1000), we observed little effect on the accuracy of Asian and Lookback options (but did significantly increase accuracy for European options). This is likely because the 'stratification' is only happening at the endpoints in our method and so will be incredibly powerful when the option payoff depends strongly on the endpoints, but less so when the endpoints are less important to the payoff (such as Asian options). Even still, it had a very similar performance to antithetic variates variance reduction and was not significantly worse even for Asian options.

Stratified sampling is a good way to reduce variance in Monte Carlo estimates but it does have some downsides. It is more complicated to implement in code, and it places more restrictions on the parameters such as the number of timesteps (due to the manner in which Brownian bridges are constructed) and the number of paths (which must be divisible by the number of strata.

Stratified sampling would also be challenging to implement for the task in part I of this project. An initial approach may be to fix the endpoints of each asset in one strata, but this is incorrect, as it is possible for different assets to end up in different strata. Instead, it would be required to have strata for each asset ending up in each division of the number line, which would end up being $n ^ M$ strata, where $M$ is the number of assets and $n$ is the number of times we divide the interval $[0, 1]$, which can very quickly grow large for bigger amounts of strata and assets.